In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import segmentation_models_pytorch as smp
from BlissLearn.utils import read_img, calculate_iou
import json
import gc

from segmentation_models_pytorch.losses import DiceLoss, FocalLoss
from BlissLearn.Core.BlissLearner import BlissLearner
from BlissLearn.Core.BlissCallbacks.Callbacks import SegmentationMetricsCallback, PrintCriteriaCallback

In [ ]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [ ]:
colors_file = r"C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\src\metadata\rgb_colors.json"
porosty_file = r"C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\src\metadata\porosty_info.json"

colors = load_json(colors_file)
porosty = load_json(porosty_file)

In [ ]:
NUM_CLASSES = len(porosty[0]['classes'])
NUM_CLASSES

In [ ]:
BATCH_SIZE = 2 
SIZE = 1024


path_to_train = r'C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\data\train-test\segmentation-train-data.xlsx'
path_to_test = r'C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\data\train-test\segmentation-test-data.xlsx'
path_to_models = r'C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\resources'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
train_df = pd.read_excel(path_to_train, index_col=0)
train_df['n'] = train_df[[f'class{i}' for i in range(NUM_CLASSES)]].sum(1)
train_df = train_df.sort_values(by='n') 
train_df.index = np.arange(len(train_df))
train_df

In [ ]:
class_counts = train_df['n'].values
sampler = WeightedRandomSampler(weights=class_counts, num_samples=len(class_counts), replacement=True)

In [ ]:
test_df = pd.read_excel(path_to_test, index_col=0)
test_df

In [ ]:
colors = np.array([
    [0, 0, 0],        # class 0
    [0, 255, 0],      # class 1
    [255, 0, 255],    # class 2
    [255, 255, 0],    # class 3
    [255, 0, 0],      # class 4
    [0, 255, 255],    # class 5
    [255, 255, 255]   # fallback for unknown class
])

def get_image_mask(mask):

    # Клипим значения классов к максимально допустимым (вдруг в маске есть класс 6+)
    mask_clipped = np.clip(mask, 0, len(colors) - 1)
    
    # Применяем векторно цвета
    return colors[mask_clipped]

In [ ]:
def transform_train(SIZE):
    return A.Compose(
        transforms=[
            A.D4(p=0.5),
            ToTensorV2(p=1)
        ]
    )

def transform_valid(SIZE):
    return A.Compose(
        transforms=[
            A.RandomCrop(SIZE, SIZE, p=1),
            ToTensorV2(p=1)
        ]
    )

def transform_test(SIZE):
    return A.Compose(
        transforms=[
            A.CenterCrop(SIZE, SIZE, p=1),
            ToTensorV2(p=1)
        ]
    )

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, df, transforms):
        super().__init__()
        self.df = df
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        image = read_img(self.df.loc[index, 'image_path'])
        mask = read_img(self.df.loc[index, 'mask_path'], rgb=False)

        transformed = self.transforms(image=image, mask=mask)
        image, mask = transformed['image'] / 255, transformed['mask']

        return image, mask

In [ ]:
train_datasets = SegmentationDataset(train_df, transforms=transform_train(SIZE))
valid_datasets = SegmentationDataset(test_df, transforms=transform_valid(SIZE))

In [ ]:
x, y = train_datasets[0]
x.shape, y.shape

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(x.permute(1,2,0).cpu().numpy());
axs[1].imshow(get_image_mask(y.cpu().numpy()));

In [ ]:
def collate_fn(batch):
    inputs, targets = zip(*batch)

    inputs = torch.stack(inputs).to(dtype=torch.float, device=device)
    targets = torch.stack(targets).to(dtype=torch.long, device=device)

    return inputs, targets

In [ ]:
train_loader = DataLoader(
    train_datasets,
    batch_size=BATCH_SIZE,
    sampler=sampler,
    collate_fn=collate_fn,
)

valid_loader = DataLoader(
    valid_datasets,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
)

In [ ]:
xb, yb = next(iter(train_loader))
len(torch.unique(yb))

In [ ]:
model = smp.Unet(
        "mit_b3",
        activation=None,
        in_channels=3,
        classes=NUM_CLASSES,
        dropout=0.5,
        decoder_attention_type='scse'
    ).to(device)

model.load_state_dict(torch.load(r"C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\resources\UnetMITSegmentationModelFT1.pkl", weights_only=True))

In [ ]:
for param in model.parameters():
    param.requires_grad = True

# for param in model.encoder.patch_embed1.proj.parameters():
#     param.requires_grad = True

In [ ]:
loss_fn = DiceLoss('multiclass', from_logits=True)

class CombinedLoss(torch.nn.Module):
    def __init__(self, dice_weight=0.5, focal_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.dice_loss = DiceLoss('multiclass', from_logits=True)
        self.focal_loss = FocalLoss('multiclass')
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
    
    def forward(self, outputs, targets):
        dice_loss = self.dice_loss(outputs, targets)
        focal_loss = self.focal_loss(outputs, targets)
        return self.dice_weight * dice_loss + self.focal_weight * focal_loss

def accuracy(yb, preds):
    preds = torch.argmax(preds, dim=1)

    return (preds == yb).float().mean().item()

metrics_callback = SegmentationMetricsCallback(
    num_classes=NUM_CLASSES,
    common_metrics={'accuracy': accuracy},
    class_metrics={"iou": calculate_iou}
)

In [ ]:
learner = BlissLearner(
    model,
    CombinedLoss(),
    torch.optim.Adam,
    dict(lr=1e-5),
    train_loader,
    valid_loader,
    batches_to_validate=146,
    callbacks=[
        metrics_callback,
        PrintCriteriaCallback()
    ],
)

In [ ]:
learner.fit(10)

In [ ]:
plt.plot(learner._callback_state.epoch_train_loss['loss'])
plt.plot(learner._callback_state.epoch_eval_loss['loss'])
plt.grid()

In [ ]:
plt.plot(learner._callback_state.epoch_train_criteria['iou_mean'])
plt.plot(learner._callback_state.epoch_eval_criteria['iou_mean'])
plt.grid()

In [ ]:
plt.plot(learner._callback_state.epoch_train_criteria['accuracy'])
plt.plot(learner._callback_state.epoch_eval_criteria['accuracy'])
plt.grid()

In [ ]:
example_inputs = torch.randn(1, 3, SIZE, SIZE).to(device)
onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)
onnx_program.optimize()
onnx_program.save(path_to_models + r"\UnetMITSegmentationModelTargetModel.onnx")

In [ ]:
torch.save(model.state_dict(), path_to_models + r"\UnetMITSegmentationModelTargetModel.pkl")

In [ ]:
# learner.get_train_info()

In [ ]:
model = smp.Unet(
        "mit_b3",
        activation=None,
        in_channels=3,
        classes=NUM_CLASSES,
        dropout=0.35,
        decoder_attention_type='scse'
    ).to(device)

# model.load_state_dict(torch.load(r"C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\resources\UnetMITSegmentationModel.pkl", weights_only=True))

model.load_state_dict(torch.load(r"C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\resources\UnetMITSegmentationModelTargetModel.pkl", weights_only=True))

In [ ]:
example_inputs = torch.randn(1, 3, 2048, 2048).to(device)
onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)
onnx_program.optimize()
onnx_program.save(path_to_models + r"\UnetMITSegmentationModelTargetModel2k.onnx")

In [ ]:
test_datasets = SegmentationDataset(test_df, transforms=transform_valid(SIZE))

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
N = len(test_datasets)
# N = 10
fig, axs = plt.subplots(N, 3, figsize=(15, 120))
model.eval()

for i, (x, y) in enumerate(test_datasets):
    with torch.inference_mode():
        inputs = torch.unsqueeze(x.to(device).float(), 0)
        preds = model(inputs)[0]


    print(preds.shape)
    img = x.permute(1, 2, 0).cpu().numpy()
    mask = y.cpu().numpy()
    pred_mask = preds.argmax(dim=0).detach().cpu().numpy()
    
    axs[i, 0].imshow(img);
    axs[i, 0].axis(False)
    axs[i, 1].imshow(get_image_mask(mask));
    axs[i, 1].axis(False)
    axs[i, 2].imshow(get_image_mask(pred_mask));
    axs[i, 2].axis(False)
    if i == N - 1:
        break

In [ ]:
# model.load_state_dict(torch.load(r"C:\Users\Viktor\Documents\IT\ReservoirRockAnalysis\resources\UnetMITSegmentationModelFT.pkl", weights_only=True))

In [ ]:
NUM_CLASSES = 6
N_ITER = 20

model.eval()

all_ious = [[] for _ in range(NUM_CLASSES)]
all_accuracies = []

with torch.inference_mode():
    for iteration in range(N_ITER):
        print(f"Validation pass {iteration + 1}/{N_ITER}...")

        for x, y_true in valid_loader:
            x = x.to(device).float()
            y_true = y_true.to(device).long()

            preds = model(x)  # (B, C, H, W)
            y_pred = preds.argmax(dim=1)  # (B, H, W)

            # Accuracy по всей маске
            batch_acc = (y_pred == y_true).float().mean().item()  # <-- передаём "сырые" logits
            all_accuracies.append(batch_acc)

            # IoU по классам
            for class_id in range(NUM_CLASSES):
                pred_class = (y_pred == class_id)
                true_class = (y_true == class_id)
                iou = calculate_iou(pred_class, true_class)
                all_ious[class_id].append(iou)

# Подсчет итоговых метрик
mean_ious = [sum(class_ious) / len(class_ious) for class_ious in all_ious]
mean_iou = sum(mean_ious) / NUM_CLASSES
mean_acc = sum(all_accuracies) / len(all_accuracies)

# Вывод
print(f"\nAggregated TTA Validation Results over {N_ITER} iterations:")
for i, iou in enumerate(mean_ious):
    print(f" IoU class_{i}: {iou:.3f}")
print(f" Mean IoU: {mean_iou:.3f}")
print(f" Mean Accuracy: {mean_acc:.3f}")